In [38]:
import os
import sys
import csv
import json
import pandas as pd
import pickle
import plotly
import shutil

from jupyter_dash import JupyterDash

import dash
import dash_core_components as dcc
import dash_html_components as html

from dash.dependencies import Input, Output, State

In [40]:
import tkinter as tk     # from tkinter import Tk for Python 3.x
from tkinter import filedialog as fd

root = tk.Tk()
root.withdraw()

FILENAME = fd.askopenfilename() # show an "Open" dialog box and return the path to the selected file
print(FILENAME)

root.quit()

shutil.copy2(FILENAME, '.')

/Users/mehtaab/APLogs/datalog2.csv


'./datalog2.csv'

In [41]:
DATALOG_FILENAME_FULL = os.path.basename(FILENAME)
DATALOG_FILENAME_BASE = DATALOG_FILENAME_FULL.split('.')[0]
DATALOG_PICKLE_NAME = DATALOG_FILENAME_BASE + '.pkl'

# file -I datalog2.csv reveals:
#    datalog2.csv: text/csv; charset=iso-8859-1
#    - this must be a COBB AP Spec, so the program handles it statically
# in practice, this encoding could be variable, 
#    thus it remains a 
# TODO: Parameterize a detected charset from the file read in
DATALOG_ENCODING = 'iso-8859-1'

RAW_DATA_FRAME = None

In [42]:
try:
    with open(DATALOG_FILENAME_FULL, 'r', encoding=DATALOG_ENCODING) as log_file:
        RAW_DATA_FRAME = pd.read_csv(log_file, index_col=0)
except Exception as e:
    print('error reading csv. probably an encoding issue.')
    print(repr(e));
    sys.exit(1)

df = RAW_DATA_FRAME

# Parse information label and drop, to begin cleaning
try:
    label = df.columns[-1]
    if label.startswith('AP Info'):
        INFO = label
        df = df.drop(columns=label)
    else:
        pass
except: 
    pass

print(INFO)

col_labels = list(df.columns)

label_and_units = [(' '.join(label.split()[0:-1]), label.split()[-1][1:-1]) for label in col_labels]
labels = [l[0] for l in label_and_units]
units  = [l[1] for l in label_and_units]
for e in zip(labels, units):
    print(e)

UNITS_BY_LABEL = dict(zip(labels, units))
                      
df = df.set_axis(labels, axis=1)
df

AP Info:[AP3-SUB-003 v1.7.4.1-17436][2014 USDM Impreza WRX STI COBB Custom Features][Reflash: Mehtaab 2014 STI AP base map.ptm - Realtime: Mehtaab 2014 STI AP base map.ptm]
('AF Correction 1', '%')
('AF Correction 3', '%')
('AF Learning 1', '%')
('AF Sens 1 Ratio', 'AFR')
('AF Sens 3 Volts', 'V')
('Accel Position', '%')
('Baro Pressure', 'psi')
('Boost', 'psi')
('Calculated Load', 'g/rev')
('Clutch Sw', 'on/off')
('Comm Fuel Final', 'AFR')
('Coolant Temp', 'F')
('Dyn Adv Mult', 'DAM')
('Dynamic Adv Lrn', '°')
('Feedback Knock', '°')
('Fine Knock Learn', '°')
('Fuel Economy', 'mpg')
('Fuel Pump Duty', '%')
('Gear Position', 'Gear')
('Idle Airflow', 'g/s')
('Idle Spd Target', 'RPM')
('Idle Speed Error', 'RPM')
('Ignition Timing', '°')
('Inj Duty Cycle', '%')
('Intake Temp', 'F')
('MAF', 'g/s')
('MAF Volts', 'V')
('Man Abs Press', 'psi')
('RPM', 'RPM')
('RPM Delta', 'RPM')
('Req Torque', 'Nm')
('SI Drive Mode', 'raw')
('TD Boost Error', 'psi')
('Target Boost Final Rel SL', 'psi')
('Thrott

,AF Correction 1,AF Correction 3,AF Learning 1,AF Sens 1 Ratio,AF Sens 3 Volts,Accel Position,Baro Pressure,Boost,Calculated Load,Clutch Sw,...,Man Abs Press,RPM,RPM Delta,Req Torque,SI Drive Mode,TD Boost Error,Target Boost Final Rel SL,Throttle Pos,Vehicle Speed,Wastegate Duty
Time (sec),,,,,,,,,,,,,,,,,,,,,
0.000,-14.06,0.0,0.0,14.59,0.38,0.0,14.36,-8.62,0.38,on,...,5.76,818,-1,0.0,3,9.17,0.23,4.7,0,0.0
0.151,-14.06,0.0,0.0,14.59,0.38,0.0,14.36,-8.60,0.38,on,...,5.74,833,1,0.0,3,9.19,0.23,4.7,0,0.0
0.324,-14.06,0.0,0.0,14.70,0.39,0.0,14.36,-8.67,0.38,on,...,5.70,832,-1,0.0,3,9.22,0.23,4.7,0,0.0
0.508,-14.06,0.0,0.0,14.59,0.39,0.0,14.36,-8.69,0.38,on,...,5.70,836,2,0.0,3,9.23,0.23,4.7,0,0.0
0.680,-14.06,0.0,0.0,14.59,0.39,0.0,14.36,-8.69,0.37,on,...,5.70,833,-5,0.0,3,9.23,0.23,4.7,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1969.695,-6.25,0.0,0.0,14.81,0.71,0.0,14.36,-9.61,0.30,off,...,4.70,800,-1,0.0,3,10.23,0.23,3.5,0,0.0
1969.872,-6.25,0.0,0.0,14.81,0.71,0.0,14.36,-9.61,0.29,off,...,4.70,813,5,0.0,3,10.23,0.23,3.5,0,0.0
1970.051,-6.25,0.0,0.0,14.70,0.71,0.0,14.36,-9.64,0.30,off,...,4.68,800,-7,0.0,3,10.24,0.23,3.5,0,0.0


In [43]:
# Set data types

DATA_TYPES = {
    'categorical' : ['SI Drive Mode', 'Clutch Sw']
}

DATA_CATEGORIES = {
    'debug' : [
        'Vehicle Speed',
        'Gear Position',
        'Throttle Pos',
        'Req Torque',
        'RPM',
        'Boost',
        'TD Boost Error',
        'Wastegate Duty'
    ],
    'inputs' : [
        'SI Drive Mode',
        'Accel Position', 
        'Throttle Pos', 
        'Gear Position',
        'Clutch Sw',
    ],
    'derived' : [
        'Calculated Load',
        'Fuel Economy',
        'RPM Delta',
        'Req Torque',
        'Vehicle Speed'
    ],
    'environmental' : [
        'Baro Pressure'
    ],
    'engine_parameters' : [
        'AF Correction 1',
        'AF Sens 1 Ratio',
        'AF Sens 3 Volts',
        'Boost',
        'Comm Fuel Final',
        'Coolant Temp',
        'Feedback Knock',
        'Fuel Pump Duty',
        'Idle Airflow',
        'Idle Speed Error',
        'Ignition Timing',
        'Inj Duty Cycle',
        'Intake Temp',
        'MAF',
        'MAF Volts',
        'Man Abs Press',
        'RPM',
        'TD Boost Error',
        'Target Boost Final Rel SL'
    ],
    'graph' : [
        'Vehicle Speed',
        'RPM',
        'Gear Position',
        'Clutch Sw',
        'Accel Position',
        'Throttle Pos',
        'Boost'
    ],
    'pressure' : [
        'RPM',
        'Boost',
        'Man Abs Press',
        'Baro Pressure'
    ],
    'defaults' : [
        'SI Drive Mode',
        'Gear Position',
        'RPM',
        'Vehicle Speed',
        'AF Correction 1',
        'AF Learning 1',
        'AF Sens 1 Ratio',
        'Accel Position',
        'Throttle Pos',
        'Baro Pressure',
        'Boost',
        'TD Boost Error',
        'Wastegate Duty',
        'Calculated Load',
        'Comm Fuel Final',
        'Coolant Temp',
        'Dyn Adv Mult',
        'Feedback Knock',
        'Fine Knock Learn',
        'Ignition Timing',
        'Inj Duty Cycle',
        'Intake Temp',
        'MAF',
        'MAF Volts',
        'Man Abs Press',
        'Req Torque'
    ]
}

DATA_CATEGORIES['driving'] = (
    DATA_CATEGORIES['inputs'] + 
    ['RPM', 'Vehicle Speed', 'Boost','Man Abs Press'] + 
    DATA_CATEGORIES['environmental']
)

In [44]:
# transform and clean data

for data in DATA_TYPES['categorical']:
    try:
        df[data] = df[data].astype('category')
    except:
        pass
try:
    CLUTCH_SW_NUM = [1 if sw == 'on' else 0 for sw in list(df['Clutch Sw'])]
    df['Clutch Sw'] = CLUTCH_SW_NUM
except:
    pass

# clean = []

# for label in labels:
#     try:
#         if (df[label].dtype != 'category'):
#             if df[label].std() == 0:
#                 clean.append(label)
#     except:
#         pass
    
# print(clean)
    
# for c in clean:
#     if df[c].nunique() == 1:
#         try:
#             df = df.drop(c, axis=1)
#         except:
#             pass

In [45]:
with open(DATALOG_FILENAME_BASE + '.pkl', 'wb') as log_pkl:
    pickle.dump(df, log_pkl)

In [46]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

TIME_INDEX = list(df.index)

fig = None

def graph_labels(labels=[], title="Parameters"):
    fig = make_subplots(rows=len(labels), cols=1, shared_xaxes=True)
    
    for i in range(0, len(labels)):
        label = to_graph[i]
        fig.add_trace(
#              plotly.graph_objs.Line is deprecated.
#             Please replace it with one of the following more specific types
#               - plotly.graph_objs.scatter.Line
#               - plotly.graph_objs.layout.shape.Line
#               - etc.
            go.Line(x=TIME_INDEX, y=list(df[label]), name=label),
            row = i+1, col = 1
        )
        
    default_plot_width = len(TIME_INDEX)/2

    fig.update_layout(template = 'plotly_dark')
    fig.update_layout(height=len(labels) * 150, width=default_plot_width, title_text=title)
    fig.update_layout(legend=dict(
        yanchor="bottom",y=-0.1,xanchor="left", x=0, orientation="h"
    ))
#     fig.update_layout(height=len(labels) * 150, title_text=title)
#     fig.update_layout(height=len(labels) * 150, title_text=title)
    app = JupyterDash()
    
    app.layout = html.Div([
        dcc.Graph(id='graph',figure=fig)])
    
    @app.callback(
        Output('graph', 'figure'),
        Input('graph', 'relayoutData'),
        State('graph', 'figure'),
        prevent_initial_call=True
    )
    def display_relayout_data(relayoutData, figure):
#         print(relayoutData)
#         print(json.dumps(figure['layout'], indent=4))
        fig = figure
        if 'xaxis.autorange' in relayoutData:
            fig['layout']['width'] = default_plot_width
        elif 'xaxis.range[0]' in relayoutData:
            range_width = relayoutData['xaxis.range[1]'] - relayoutData['xaxis.range[0]']
            fig['layout']['width'] = 1000

        figure = fig
        return figure


    
    app.run_server(mode='inline', port = 8090, dev_tools_ui=True, debug=True,
              dev_tools_hot_reload =True, threaded=True)
        
#     
#     fig.show()

# to_graph = DATA_CATEGORIES['driving']

# graph_labels(to_graph, 'Driving')


# to_graph = DATA_CATEGORIES['derived'] + DATA_CATEGORIES['engine_parameters']

# graph_labels(to_graph, 'Engine')

to_graph = DATA_CATEGORIES['debug']

graph_labels(to_graph, 'Graph')

# to_graph = DATA_CATEGORIES['pressure']

# graph_labels(to_graph, 'Pressure')




# app.run_server(debug=True, use_reloader=False, mode='inline')  # Turn off reloader if inside Jupyter


/opt/homebrew/Caskroom/miniconda/base/envs/log-dive/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [103]:
for k, v in UNITS_BY_LABEL.items():
    print(k, ':', v)

AF Correction 1 : %
AF Correction 3 : %
AF Learning 1 : %
AF Sens 1 Ratio : AFR
AF Sens 3 Volts : V
Accel Position : %
Baro Pressure : psi
Boost : psi
Calculated Load : g/rev
Clutch Sw : on/off
Comm Fuel Final : AFR
Coolant Temp : F
Dyn Adv Mult : DAM
Dynamic Adv Lrn : °
Feedback Knock : °
Fine Knock Learn : °
Fuel Economy : mpg
Fuel Pump Duty : %
Gear Position : Gear
Idle Airflow : g/s
Idle Spd Target : RPM
Idle Speed Error : RPM
Ignition Timing : °
Inj Duty Cycle : %
Intake Temp : F
MAF : g/s
MAF Volts : V
Man Abs Press : psi
RPM : RPM
RPM Delta : RPM
Req Torque : Nm
SI Drive Mode : raw
TD Boost Error : psi
Target Boost Final Rel SL : psi
Throttle Pos : %
Vehicle Speed : mph
Wastegate Duty : %


In [104]:
secs = list(df.index)
gear = list(df['Gear Position'])
clsw = list(df['Clutch Sw'])
rpm = list(df['RPM'])
speed = list(df['Vehicle Speed'])
acc = list(df['Accel Position'])

shift_attrs = {
    'TIME_START' : 0.0,
    'TIME_END' : 0.0,
    'GEAR_START' : 0,
    'GEAR_END' : 0,
    'RPM_START' : 0,
    'RPM_END' : 0,
    'SPEED_START' : 0,
    'SPEED_END' : 0,
    'ACC_START' : 0,
    'ACC_END' : 0,
}
SHIFT_PARAMS = [
    'TIME',
    'GEAR',
    'RPM',
    'SPEED',
    'ACC'
]
attr_map = {
    'TIME' : secs,
    'GEAR' : gear,
    'RPM' : rpm,
    'SPEED' : speed,
    'ACC' : acc
}

class shift(object):
    def __init__(self, args):
        for k, v in args.items():
            setattr(self, k, v)
            
    def __getitem__(self, item):
        return getattr(self, item)
    
    def __repr__(self):
        out_str = []
        for a in shift_attrs.keys():
            out_str.append(str(self[a]))
        return ' '.join(out_str)
    
    def set_values(self, val_dict):
        for k, v in val_dict.items():
            setattr(self, k, v)
        

SHIFTS = []

MAX_IDX = len(secs) - 1

CLSW_CURR = clsw[0]

SHIFT = shift(shift_attrs)
for i in range(1, MAX_IDX + 1):
    CLSW = clsw[i]
    if CLSW == 1:
        if CLSW_CURR == 0:
            # clutch in
            start_dict = {}
            for param in SHIFT_PARAMS:
                param_string = param + '_START'
                start_dict[param_string] = attr_map[param][i]
            print(json.dumps(start_dict))
            SHIFT.set_values(start_dict)
        else:
            continue
    if CLSW == 0:
        if CLSW_CURR == 1:
            end_dict = {}
            for param in SHIFT_PARAMS:
                param_string = param + '_END'
                end_dict[param_string] = attr_map[param][i]
            print(json.dumps(end_dict))
            SHIFT.set_values(end_dict)
            SHIFTS.append(SHIFT)
            SHIFT = shift(shift_attrs)
        else:
            continue
    CLSW_CURR = CLSW
        
for s in SHIFTS:
    out_str = []
    for attr in shift_attrs:
        out_str.append(str(s[attr]))
    print(' '.join(out_str))

{"TIME_END": 35.552, "GEAR_END": 0, "RPM_END": 721, "SPEED_END": 0, "ACC_END": 0.0}
{"TIME_START": 61.749, "GEAR_START": 0, "RPM_START": 798, "SPEED_START": 0, "ACC_START": 0.0}
{"TIME_END": 85.993, "GEAR_END": 0, "RPM_END": 872, "SPEED_END": 3, "ACC_END": 0.0}
{"TIME_START": 107.598, "GEAR_START": 0, "RPM_START": 852, "SPEED_START": 0, "ACC_START": 0.0}
{"TIME_END": 161.054, "GEAR_END": 0, "RPM_END": 914, "SPEED_END": 7, "ACC_END": 0.0}
{"TIME_START": 165.478, "GEAR_START": 0, "RPM_START": 899, "SPEED_START": 4, "ACC_START": 0.0}
{"TIME_END": 174.326, "GEAR_END": 1, "RPM_END": 1101, "SPEED_END": 6, "ACC_END": 8.6}
{"TIME_START": 177.508, "GEAR_START": 1, "RPM_START": 2334, "SPEED_START": 12, "ACC_START": 0.0}
{"TIME_END": 179.808, "GEAR_END": 2, "RPM_END": 1494, "SPEED_END": 12, "ACC_END": 11.0}
{"TIME_START": 186.181, "GEAR_START": 2, "RPM_START": 2636, "SPEED_START": 23, "ACC_START": 0.0}
{"TIME_END": 187.952, "GEAR_END": 3, "RPM_END": 1736, "SPEED_END": 22, "ACC_END": 0.8}
{"TIME_S

In [105]:
# Shift Points
GEARS = list(range(1,7))
GEAR_RATIOS = [0, 3.636, 2.235, 1.521, 1.137, 0.971, 0.756]
RPMS_IDX = list(range(0, 8100))
TRANS_OUTPUT_RPM = pd.DataFrame(index=RPMS_IDX, columns=GEARS)

for index, row in TRANS_OUTPUT_RPM.iterrows():
    for GEAR in GEARS:
        row[GEAR] = int(round(index / GEAR_RATIOS[GEAR]))
        
TRANS_OUTPUT_RPM

,1,2,3,4,5,6
0,0,0,0,0,0,0
1,0,0,1,1,1,1
2,1,1,1,2,2,3
3,1,1,2,3,3,4
4,1,2,3,4,4,5
...,...,...,...,...,...,...
8095,2226,3622,5322,7120,8337,10708
8096,2227,3622,5323,7120,8338,10709
8097,2227,3623,5323,7121,8339,10710
8098,2227,3623,5324,7122,8340,10712


In [106]:
# MECHANICALLY CALCULATED SPEED
FINAL_DRIVE = 3.9
WHEEL_CIRCUMFERENCE = 80.7
MECH_CALC_SPEED = pd.DataFrame(index=RPMS_IDX, columns=GEARS)
for index, row in MECH_CALC_SPEED.iterrows():
    for GEAR in GEARS:
        row[GEAR] = round((TRANS_OUTPUT_RPM.loc[index, GEAR] / FINAL_DRIVE) * (WHEEL_CIRCUMFERENCE / 1056), 3)

MECH_CALC_SPEED

,1,2,3,4,5,6
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.02,0.02,0.02,0.02
2,0.02,0.02,0.02,0.039,0.039,0.059
3,0.02,0.02,0.039,0.059,0.059,0.078
4,0.02,0.039,0.059,0.078,0.078,0.098
...,...,...,...,...,...,...
8095,43.618,70.973,104.285,139.516,163.363,209.823
8096,43.638,70.973,104.304,139.516,163.383,209.843
8097,43.638,70.993,104.304,139.536,163.403,209.862
8098,43.638,70.993,104.324,139.556,163.422,209.902


In [114]:
# Shift RPM Calculator
def shift_RPM_calculate(shift):
    TRANS_RPM_START = TRANS_OUTPUT_RPM.loc[shift.RPM_START, shift.GEAR_START]
    END_TRANS_RPMS = list(TRANS_OUTPUT_RPM[shift.GEAR_END])
    END_ENGINE_RPM_IDEAL = 0
    DELTAS = [ abs(val - TRANS_RPM_START) for val in END_TRANS_RPMS ]
    
    LOWEST_DELTA = sys.maxsize
    for idx, delta in enumerate(DELTAS):
        if delta <= LOWEST_DELTA:
            LOWEST_DELTA = delta
            END_ENGINE_RPM_IDEAL = idx
        
    return END_ENGINE_RPM_IDEAL

In [120]:
SHIFT_LISTS = {
    "0,1" : [],
    "1,2" : [],
    "2,3" : [],
    "3,4" : [],
    "4,5" : [],
    "5,6" : []
} 

for s in SHIFTS:        
    UP_OR_DOWN = "UP" if s.GEAR_END > s.GEAR_START else "DOWN"
    SHIFT_LEN = s.TIME_END - s.TIME_START
    
    SHIFT_STRING = f'{s.TIME_START:>8.2f} {s.TIME_END:>7.2f} [ {SHIFT_LEN:5.2f} ]'
    GEAR_STRING = f'{s.GEAR_START:>{s.GEAR_END*4}d}->{s.GEAR_END}'
    
    SPEED_STRING = f'{s.SPEED_START:>3} {s.SPEED_END:>3}'

    if s.GEAR_START != 0 and s.GEAR_END != 0:
        IDEAL_END_RPM = shift_RPM_calculate(s)
        RPM_DELTA = s.RPM_END - IDEAL_END_RPM
        RPM_STRING = f'{s.RPM_START:>4} {s.RPM_END:>4} {IDEAL_END_RPM:>4} {RPM_DELTA:>5}'
        print(f'{SHIFT_STRING:<22} {GEAR_STRING:<{28}} {RPM_STRING:<21} {SPEED_STRING:<6}')
    else:
        print(f'{SHIFT_STRING:<22} {GEAR_STRING:<{28}}')
    
    if s.GEAR_END - 1 == s.GEAR_START:
        SHIFT_LISTS[f'{s.GEAR_START},{s.GEAR_END}'].append(s)
    

    0.00   35.55 [ 35.55 ] 0->0                        
   61.75   85.99 [ 24.24 ] 0->0                        
  107.60  161.05 [ 53.46 ] 0->0                        
  165.48  174.33 [  8.85 ]    0->1                     
  177.51  179.81 [  2.30 ]        1->2                  2334 1494 1435    59   12  12
  186.18  187.95 [  1.77 ]            2->3              2636 1736 1794   -58   23  22
  193.26  194.14 [  0.88 ] 3->0                        
  197.16  199.99 [  2.83 ]        0->2                 
  206.18  207.77 [  1.59 ]            2->3              2808 1840 1911   -71   24  24
  210.61  211.50 [  0.89 ] 3->0                        
  212.38  215.74 [  3.36 ]        0->2                 
  219.63  221.93 [  2.30 ]            2->3              2647 1793 1801    -8   23  22
  227.96  229.54 [  1.59 ]                3->4          2468 1816 1845   -29   32  30
  256.81  257.51 [  0.70 ] 4->0                        
  259.29  261.40 [  2.12 ]            0->3             
  267.78  

In [52]:
from statistics import fmean, quantiles

for k,v in SHIFT_LISTS.items():
    print('--', k, '--')
    SHIFT_TIMES = [S[1] - S[0] for S in v]
    print("mean:", fmean(SHIFT_TIMES))
    print("quantiles:", quantiles(SHIFT_TIMES))
    print('')

-- 0,1 --
mean: 7.005
quantiles: [3.718999999999994, 7.4360000000000355, 8.847999999999985]

-- 1,2 --
mean: 2.0172000000000194
quantiles: [1.7715000000000316, 2.1200000000000045, 2.211500000000015]

-- 2,3 --
mean: 1.8228999999999957
quantiles: [1.7230000000000203, 1.7709999999999724, 1.9965000000000543]

-- 3,4 --
mean: 1.662700000000021
quantiles: [1.4190000000000254, 1.5905000000000769, 1.808499999999981]

-- 4,5 --
mean: 1.9445000000000263
quantiles: [1.7647499999999923, 1.9475000000000193, 2.1234999999999786]

-- 5,6 --
mean: 1.6160000000000017
quantiles: [1.4110000000000014, 1.5909999999998945, 1.9519999999999982]

